In [5]:
#generates list of users to check
import os
dpath = 'demoji_diagnosed/'
upath = 'demoji_undiagnosed/'

unlist = os.listdir(upath)
dlist = os.listdir(dpath)

danxiety_users = []
uanxiety_users = []
anxiety_members = []

#anxiety members
afile = open('anxiety_members2.txt', 'r')
for member in afile:
    anxiety_members.append(member.strip())
afile.close()

#undiagnosed list
for file in unlist:
    if '_submission' in file:
        name = file.split('_submission')[0]
        if not name in uanxiety_users:
            if name in anxiety_members:
                uanxiety_users.append(name)
    elif '_comment' in file:
        name = file.split('_comment')[0]
        if not name in uanxiety_users:
            if name in anxiety_members:
                uanxiety_users.append(name)



            
#diagnosed list
for file in dlist:
    if '_submission' in file:
        name = file.split('_submission')[0]
        if not name in danxiety_users:
            if name in anxiety_members:
                danxiety_users.append(name)
    elif '_comment' in file:
        name = file.split('_comment')[0]
        if not name in danxiety_users:
            if name in anxiety_members:
                danxiety_users.append(name)
            

overlap_members = []#should ideally remain empty
for person in uanxiety_users:
    if person in danxiety_users:
        overlap_members.append(person)

In [18]:
#length sanity check
print(f'Undiagnosed users: {len(uanxiety_users)}')
print(f'Diagnosed users: {len(danxiety_users)}')
print(f'Overlapped users: {len(overlap_members)}')

Undiagnosed users: 1869
Diagnosed users: 675
Overlapped users: 0


In [72]:
#writing anxiety members to file
unfile = open('proven_anxiety_undiagnosed.txt','w')
dfile = open('proven_anxiety_diagnosed.txt','w')
for user in uanxiety_users:
    unfile.write(user)
    unfile.write('\n')
for user in danxiety_users:
    dfile.write(user)
    dfile.write('\n')
unfile.close()
dfile.close()

In [80]:
import re

diagnose_file = open('diagnose_regex.txt', 'r')
diagnose_list = []
all_diagnosed = [] #stores all diagnosed members regardless of subreddit
all_undiagnosed = [] #stores all undiagnosed members regardless of subreddit
for line in diagnose_file:
    d = line.strip()
    diagnose_list.append(d)
diagnose_file.close()
dproven_users = []
uproven_users = []
dsubs = {}
usubs = {}

for user in danxiety_users:
   
   
    cfilename = 'demoji_diagnosed/' + user + '_comments_stripped.txt'
    sfilename = 'demoji_diagnosed/' + user + '_submissions_stripped.txt'
    cfile = open(cfilename,'r')
    sfile = open(sfilename, 'r')
    for line in sfile:
     
        linelist = line.split(':;')
        if len(linelist) != 15:
            dskipped += 1
            continue
        selftext = linelist[9]
        selftext += ' '
        selftext += linelist[14]
        
        subreddit = linelist[10]
       # if not 'anxiety' in subreddit:
          #  continue
        selftext = selftext.lower()
        for phrase in diagnose_list:
            if re.search(phrase, selftext) != None:
            
               # print(phrase)
                if subreddit not in dsubs:
                    dsubs[subreddit] = 0
                else:
                    dsubs[subreddit] += 1
                if not user in dproven_users:
                    dproven_users.append(user)
              
    
    for line in cfile:
        
        linelist = line.split(':;')
        if len(linelist) != 12:
           
            continue
        body = linelist[2] 
        subreddit = linelist[9]
       # if not 'anxiety' in subreddit:
         #   continue
       
        body = body.lower()
        for phrase in diagnose_list:
            if re.search(phrase, body) != None:
                if subreddit not in dsubs:
                    dsubs[subreddit] = 0
                else:
                    dsubs[subreddit] += 1
               # print(phrase)
                if not user in dproven_users:
                    dproven_users.append(user)
              
    cfile.close()
    sfile.close()
 


for user in uanxiety_users:
    
    
    cfilename = 'demoji_undiagnosed/' + user + '_comments_stripped.txt'
    sfilename = 'demoji_undiagnosed/' + user + '_submissions_stripped.txt'
   
    cfile = open(cfilename,'r')
    sfile = open(sfilename, 'r')
    for line in sfile:
       
        linelist = line.split(':;')
        if len(linelist) != 15:
            
            continue
        selftext = linelist[9]
        selftext += ' '
        selftext += linelist[14]
        
        subreddit = linelist[10]
       # if not 'anxiety' in subreddit:
           # continue
        selftext = selftext.lower()
        for phrase in diagnose_list:
            
            if re.search(phrase, selftext) != None:
                if subreddit not in usubs:
                    usubs[subreddit] = 0
                else:
                    usubs[subreddit] += 1
               # print(phrase)
                if not user in uproven_users:
                    uproven_users.append(user)

    for line in cfile:

        linelist = line.split(':;')
        if len(linelist) != 12:
            
            continue
        body = linelist[2] 
        subreddit = linelist[9]
        if not 'anxiety' in subreddit.lower():
            continue
        body = body.lower()
        for phrase in diagnose_list:
            if re.search(phrase, body) != None:
                if subreddit not in usubs:
                    usubs[subreddit] = 0
                else:
                    usubs[subreddit] += 1
               # print(phrase)
                if not user in uproven_users:
                    uproven_users.append(user)
             
    cfile.close()
    sfile.close()
   
print(f'len diag {len(danxiety_users)}')
print(f'len undiag: {len(uanxiety_users)}')


len diag 675
len undiag: 1869


In [81]:
from operator import itemgetter
res = dict(sorted(dsubs.items(), key = itemgetter(1), reverse = True))
pfile = open('dsubs_where_diagnosed.txt','w')
for key in res:
    pfile.write(str(key))
    pfile.write(';')
    pfile.write(str(res[key]))
    pfile.write('\n')
pfile.close()

res = dict(sorted(usubs.items(), key = itemgetter(1), reverse = True))
pfile = open('usubs_where_diagnosed.txt','w')
for key in res:
    pfile.write(str(key))
    pfile.write(';')
    pfile.write(str(res[key]))
    pfile.write('\n')
pfile.close()

In [82]:
#sanity check 2
print(f'Undiagnosed users: {len(uproven_users)}')
print(f'Diagnosed users: {len(dproven_users)}')
print(f'Diagnosis criteria length: {len(diagnose_list)}')


Undiagnosed users: 285
Diagnosed users: 454
Diagnosis criteria length: 145


In [ ]:
import matplotlib.pyplot as pt
import re

diagnose_file = open('diagnose_regex.txt', 'r')
diagnose_list = []
all_diagnosed = [] #stores all diagnosed members regardless of subreddit
all_undiagnosed = [] #stores all undiagnosed members regardless of subreddit
for line in diagnose_file:
    d = line.strip()
    diagnose_list.append(d)
diagnose_file.close()

diagnosed_members = {}
undiagnosed_members = {}
diagnose_posts = {}

subred_list = ['anxiety','Anxietyhelp','depressed', 'EOOD', 'depression_help', 'mentalhealth', 'socialanxiety', 'ADHD', 'schizophrenia', 'depression', 'depressionregimens']

members = {}

#compile lists of users for each sub (can reuse code)
for sub in subred_list:
    diagnose_posts[sub] = {}
    diagnosed_members[sub] = []
    sub_member_fname = sub + '_members2.txt'
    member_file = open(sub_member_fname, 'r')
    
    #populates member dictionary for this sub
    for line in member_file:
        member = line.strip()
        if member == 'Author':
            continue
        if sub in members:
            if member in members[sub]:
                continue
            else:
                members[sub].append(member)
          
        else:
            members[sub] = []
            members[sub].append(member)
    member_file.close()
    
    
    #up to here - 11/15
    #print(sub + ':' + str(len(members[sub])))
    
    #looping through submission files
    sub_filename = sub + '_submissions.txt'
    sub_file = open(sub_filename, 'r')
    for submission in sub_file:
        sub_list = submission.strip().split(',')
        
        #valid submissions have 8 fields
        if len(sub_list) == 8:
            temp_member = sub_list[3]
            if temp_member in members[sub]:
                #if the member has already been listed as diagnosed, we skip them 
                #(should we list diagnosed people for each sub? we have no way of programmatically knowing what they are diagnosed with at this time)
                #i have decided that we should because we can always remove duplicates later if we want
                if temp_member in diagnosed_members[sub]:
                    continue
                else:
                    post_string = sub_list[1] + sub_list[2] #string to be processed CONVERT TO LOWERCASE
                    post_string = post_string.lower()
                    #add code here
                    for phrase in diagnose_list:
                        if re.search(phrase, post_string) != None:
                            diagnosed_members[sub].append(temp_member)
                            diagnose_posts[sub][temp_member] = post_string
                            all_diagnosed.append(temp_member)
                            break
    sub_file.close()
    
    #looping through comment files
    com_filename = sub + '_comments.txt'
    com_file = open(com_filename, 'r')
    
    for comment in com_file:
        com_list = comment.strip().split(',')
        if(len(com_list)) ==7:
            temp_member = com_list[2]
            if temp_member in members[sub]:
                if temp_member in diagnosed_members[sub]:
                    continue
                else:
                    post_string = com_list[1]
                    post_string = post_string.lower()
                    for phrase in diagnose_list:
                        if re.search(phrase, post_string) != None:
                            diagnosed_members[sub].append(temp_member)
                            diagnose_posts[sub][temp_member] = post_string
                            all_diagnosed.append(temp_member)
                            break    
    com_file.close()
           
print('done')